### node name

In [7]:
import tensorflow as tf
from tensorflow.python.platform import gfile
GRAPH_PB_PATH = '/dataDisk/myfloder/jupyter/pb/retinanet123.pb'
with tf.Session() as sess:
   print("load graph")
   with gfile.FastGFile(GRAPH_PB_PATH,'rb') as f:
       graph_def = tf.GraphDef()
   graph_def.ParseFromString(f.read())
   sess.graph.as_default()
   tf.import_graph_def(graph_def, name='')
   graph_nodes=[n for n in graph_def.node]
   names = []
   for t in graph_nodes:
      names.append(t.name)
   print(names)

load graph
['input_1', 'Variable', 'Variable/read', 'Variable_1', 'Variable_1/read', 'Variable_2', 'Variable_2/read', 'Variable_3', 'Variable_3/read', 'Variable_4', 'Variable_4/read', 'pyramid_regression_0/kernel', 'pyramid_regression_0/kernel/read', 'pyramid_regression_0/bias', 'pyramid_regression_0/bias/read', 'pyramid_regression_1/kernel', 'pyramid_regression_1/kernel/read', 'pyramid_regression_1/bias', 'pyramid_regression_1/bias/read', 'pyramid_regression_2/kernel', 'pyramid_regression_2/kernel/read', 'pyramid_regression_2/bias', 'pyramid_regression_2/bias/read', 'pyramid_regression_3/kernel', 'pyramid_regression_3/kernel/read', 'pyramid_regression_3/bias', 'pyramid_regression_3/bias/read', 'pyramid_regression/kernel', 'pyramid_regression/kernel/read', 'pyramid_regression/bias', 'pyramid_regression/bias/read', 'pyramid_classification_0/kernel', 'pyramid_classification_0/kernel/read', 'pyramid_classification_0/bias', 'pyramid_classification_0/bias/read', 'pyramid_classification_1/ke

https://zhuanlan.zhihu.com/p/47649285

https://blog.csdn.net/u014432647/article/details/75276718

### pb file加载模型

In [8]:
import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.python.framework import graph_util
from skimage import io, transform

In [9]:
def recognize(jpg_path, pb_file_path):
    with tf.Graph().as_default():
        output_graph_def = tf.GraphDef()

        with open(pb_file_path, "rb") as f:
            output_graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(output_graph_def, name="")

        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)

            input_x = sess.graph.get_tensor_by_name("input_1:0")
            print(input_x)
            
            out_1 = sess.graph.get_tensor_by_name("output_1:0")
            print(out_1)
            
            out_2 = sess.graph.get_tensor_by_name("output_2:0")
            print(out_2)
            
            out_3 = sess.graph.get_tensor_by_name("output_3:0")
            print(out_3)     

            img = io.imread(jpg_path)
#           img = transform.resize(img, (224, 224, 3))
            img_out_1 = sess.run(out_1, feed_dict={input_x:np.expand_dims(img, axis=0)})
            print("img_out_1:",img_out_1)
#           prediction_labels = np.argmax(img_out_softmax, axis=1)
#           print("label:",prediction_labels)
            img_out_2 = sess.run(out_2, feed_dict={input_x:np.expand_dims(img, axis=0)})
            print("img_out_2:",img_out_2)
       
            img_out_3 = sess.run(out_3, feed_dict={input_x:np.expand_dims(img, axis=0)})
            print("img_out_3:",img_out_3)

In [10]:
pb_file_path='/dataDisk/myfloder/jupyter/pb/retinanet123.pb'
jpg_path='/dataDisk/myfloder/jupyter/img/Plat6_dinner_20181004_16369_NS (1).jpg'
recognize(jpg_path, pb_file_path)

Tensor("input_1:0", shape=(?, ?, ?, 3), dtype=float32)
Tensor("output_1:0", dtype=float32)
Tensor("output_2:0", dtype=float32)
Tensor("output_3:0", dtype=int32)
img_out_1: [[[307.99734 364.17786 515.4785  562.44885]
  [310.81247 538.2744  504.75415 714.8896 ]
  [307.99734 364.17786 515.4785  562.44885]
  ...
  [ -1.       -1.       -1.       -1.     ]
  [ -1.       -1.       -1.       -1.     ]
  [ -1.       -1.       -1.       -1.     ]]]
img_out_2: [[ 0.52586305  0.17236689  0.1669985   0.16121933  0.1420514   0.1215702
   0.11845362  0.11160436  0.11050129  0.10757026  0.1008617   0.09458178
   0.09341437  0.0907872   0.09030774  0.08844861  0.08376783  0.08139518
   0.07379287  0.07297677  0.07078519  0.07064423  0.06763709  0.06672558
   0.06533501  0.06355828  0.06333712  0.06129557  0.06089121  0.05973858
   0.05901104  0.05878055  0.05758283  0.05735633  0.05452091  0.05366752
   0.05325428  0.05298209  0.05260333  0.0522444   0.05179524  0.05144489
   0.0501523  -1.         -1

### no apply

In [ ]:
def recognize(jpg_path, pb_file_path):
    with tf.Graph().as_default():
        output_graph_def = tf.GraphDef()

        with open(pb_file_path, "rb") as f:
            output_graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(output_graph_def, name="")

        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)

            input_x = sess.graph.get_tensor_by_name("input_1:0")
            print(input_x)
            out_softmax = sess.graph.get_tensor_by_name("output_2:0")
            print(out_softmax)
            out_label = sess.graph.get_tensor_by_name("output_1:0")
            print(out_label)

            img = io.imread(jpg_path)
            img = transform.resize(img, (224, 224, 3))
            img_out_softmax = sess.run(out_softmax, feed_dict={input_x:np.reshape(img, [-1, 224, 224, 3])})

            print("img_out_softmax:",img_out_softmax)
            prediction_labels = np.argmax(img_out_softmax, axis=1)
            print("label:",prediction_labels)